# 基于langchain创建自己专属的对话大模型

1. 领域精准问答
2. 数据更新频繁
3. 生成内容可解释可追溯
4. 数据隐私保护

通过这个例子，我们将基于`LangChain`, `OpenAI(LLM)`,  `vector DB`构建一个属于自己的LLM模型。

主要使用的技术————***Retrieval Augmented Generation (RAG)***

首先确保自己拥有一个 `OpenAI API key` (也并非必须)


### 准备环境

In [1]:
! pip install -qU \
    langchain==0.0.316 \
    openai==0.28.1  \
    tiktoken==0.5.1  \
    cohere \
    chromadb==0.4.15

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### 创建一个对话模型(no RAG)

In [2]:
import os
from langchain.chat_models import ChatOpenAI
from getpass import getpass

OPENAI_API_KEY = getpass()

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

OpenAI Python 的例子
```python
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Knock knock."},
    {"role": "assistant", "content": "Who's there?"},
    {"role": "user", "content": "Orange."},
]
```
https://cookbook.openai.com/examples/how_to_format_inputs_to_chatgpt_models


但是langchain 需要使用以下的格式

In [3]:

from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)


messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Knock knock."),
    AIMessage(content="Who's there?"),
    HumanMessage(content="Orange"),

]

In [4]:
res = chat(messages)
res

AIMessage(content='Orange who?')

因为 `res`也是`AIMessage`属性，所以我们可以直接进行添加，即可实现下一次的响应

In [5]:
messages.append(res)
res = chat(messages)

print(res.content)

Orange you glad I'm here to help you?


#### 处理LLM存在的缺陷
1. 容易出现幻觉
2. 信息滞后
3. 专业领域深度知识匮乏


In [6]:
messages = [
    SystemMessage(content="你是一个专业的知识助手。"),
    HumanMessage(content="你知道baichuan2模型吗？"),
]

In [7]:
res = chat(messages)
print(res.content)


对不起，我不熟悉"baichuan2"模型。请问这是一个什么样的模型？可能我可以帮助您查找相关信息。


chatgpt AI无法满足我们在某些特定领域的专业需求，我们可以通过知识注入的方式，利用prompt来解决这种问题：

In [8]:

baichuan2_information = [
    "Baichuan 2是一个大规模多语言语言模型，它专注于训练在多种语言中表现优异的模型，包括不仅限于英文。这使得Baichuan 2在处理各种语言的任务时能够取得显著的性能提升。",
    "Baichuan 2是从头开始训练的，使用了包括了2.6万亿个标记的庞大训练数据集。相对于以往的模型，Baichuan 2提供了更丰富的数据资源，从而能够更好地支持多语言的开发和应用。",
    "Baichuan 2不仅在通用任务上表现出色，还在特定领域（如医学和法律）的任务中展现了卓越的性能。这为特定领域的应用提供了强有力的支持。"
]

source_knowledge = "\n".join(baichuan2_information)

In [9]:
print(source_knowledge)


Baichuan 2是一个大规模多语言语言模型，它专注于训练在多种语言中表现优异的模型，包括不仅限于英文。这使得Baichuan 2在处理各种语言的任务时能够取得显著的性能提升。
Baichuan 2是从头开始训练的，使用了包括了2.6万亿个标记的庞大训练数据集。相对于以往的模型，Baichuan 2提供了更丰富的数据资源，从而能够更好地支持多语言的开发和应用。
Baichuan 2不仅在通用任务上表现出色，还在特定领域（如医学和法律）的任务中展现了卓越的性能。这为特定领域的应用提供了强有力的支持。


In [10]:
query = "你知道baichuan2模型吗？"

prompt_template = f"""基于以下内容回答问题：

内容:
{source_knowledge}

Query: {query}"""

In [11]:
prompt = HumanMessage(
    content=prompt_template
)
messages.append(prompt)

res = chat(messages)

In [12]:
print(res.content)


Baichuan 2是一个大规模多语言语言模型，专注于训练在多种语言中表现优异的模型。它从头开始训练，使用了一个包含2.6万亿个标记的巨大训练数据集。Baichuan 2相对于以往的模型提供了更丰富的数据资源，使其能够更好地支持多语言的开发和应用。此外，Baichuan 2不仅在通用任务上表现出色，还在特定领域（如医学和法律）的任务中展现了卓越的性能，为特定领域的应用提供了强有力的支持。


当我们注入一些专业的知识后，模型就能够很好的回答相关问题。
如果每一个问题都去用相关的外部知识进行增强拼接的话，那么回答的准确性就大大增加？？？？

### 创建一个RAG对话模型

#### 1. 加载数据 （以baichuan2论文为例）

   https://arxiv.org/pdf/2309.10305v2.pdf

In [13]:
! pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.9 MB/s eta 0:00:00


In [14]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://arxiv.org/pdf/2309.10305.pdf")

pages = loader.load_and_split()

In [15]:
pages[0]

Document(page_content='Baichuan 2: Open Large-scale Language Models\nAiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan\nDian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai\nGuosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji\nJian Xie, Juntao Dai, Kun Fang, Lei Su, Liang Song, Lifeng Liu, Liyun Ru, Luyao Ma\nMang Wang, Mickel Liu, MingAn Lin, Nuolan Nie, Peidong Guo, Ruiyang Sun\nTao Zhang, Tianpeng Li, Tianyu Li, Wei Cheng, Weipeng Chen, Xiangrong Zeng\nXiaochuan Wang, Xiaoxi Chen, Xin Men, Xin Yu, Xuehai Pan, Yanjun Shen, Yiding Wang\nYiyu Li, Youxin Jiang, Yuchen Gao, Yupeng Zhang, Zenan Zhou, Zhiying Wu\nBaichuan Inc.\nAbstract\nLarge language models (LLMs) have\ndemonstrated remarkable performance on\na variety of natural language tasks based\non just a few examples of natural language\ninstructions, reducing the need for extensive\nfeature engineering. However, most powerful\nLLMs are closed-source

#### 2. 知识切片 将文档分割成均匀的块。每个块是一段原始文本

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
)

docs = text_splitter.split_documents(pages)

In [17]:
len(docs)



215

#### 3. 利用embedding模型对每个文本片段进行向量化，并储存到向量数据库中

In [18]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma


embed_model = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=docs, embedding=embed_model , collection_name="openai_embed")

#### 4. 通过向量相似度检索和问题最相关的K个文档。

In [19]:
query = "How large is the baichuan2 vocabulary?"
result = vectorstore.similarity_search(query ,k = 2)

In [20]:
result

[Document(page_content='languages, such as Chinese.\nIn this technical report, we introduce Baichuan\n2, a series of large-scale multilingual language\nmodels. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters. Both\nmodels were trained on 2.6 trillion tokens, which\nto our knowledge is the largest to date, more than\ndouble that of Baichuan 1 (Baichuan, 2023b,a).\nWith such a massive amount of training data,', metadata={'page': 1, 'source': '/tmp/tmphxnlz_0y/tmp.pdf'}),
 Document(page_content='Baichuan 2: Open Large-scale Language Models\nAiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan\nDian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai\nGuosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji\nJian Xie, Juntao Dai, Kun Fang, Lei Su, Liang Song, Lifeng Liu, Liyun Ru, Luyao Ma\nMang Wang, Mickel Liu, MingAn Lin, Nuolan Nie, Peidong G

#### 5. 原始`query`与检索得到的文本组合起来输入到语言模型，得到最终的回答

In [21]:
def augment_prompt(query: str):
  # 获取top3的文本片段
  results = vectorstore.similarity_search(query, k=3)
  source_knowledge = "\n".join([x.page_content for x in results])
  # 构建prompt
  augmented_prompt = f"""Using the contexts below, answer the query.

  contexts:
  {source_knowledge}

  query: {query}"""
  return augmented_prompt

In [22]:
print(augment_prompt(query))

Using the contexts below, answer the query.

  contexts:
  languages, such as Chinese.
In this technical report, we introduce Baichuan
2, a series of large-scale multilingual language
models. Baichuan 2 has two separate models,
Baichuan 2-7B with 7 billion parameters and
Baichuan 2-13B with 13 billion parameters. Both
models were trained on 2.6 trillion tokens, which
to our knowledge is the largest to date, more than
double that of Baichuan 1 (Baichuan, 2023b,a).
With such a massive amount of training data,
Baichuan 2: Open Large-scale Language Models
Aiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan
Dian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai
Guosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji
Jian Xie, Juntao Dai, Kun Fang, Lei Su, Liang Song, Lifeng Liu, Liyun Ru, Luyao Ma
Mang Wang, Mickel Liu, MingAn Lin, Nuolan Nie, Peidong Guo, Ruiyang Sun
adequate training of each word embedding. We
have 

In [23]:
# 创建prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)

messages.append(prompt)

res = chat(messages)

print(res.content)

Baichuan 2 has a vocabulary size of 125,696.


### 没有OPENAI api key怎么办 创建一个非openai的对话模型  


1.   embedding模型  
2.   chat模型


In [24]:
! pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [25]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

model_name = "sentence-transformers/sentence-t5-large"

In [26]:
embedding = HuggingFaceEmbeddings(model_name=model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

In [27]:
vectorstore_hf = Chroma.from_documents(documents=docs, embedding=embedding , collection_name="huggingface_embed")

In [28]:
query = "How large is the baichuan2 vocabulary?"

result = vectorstore_hf.similarity_search(query ,k = 2)


In [29]:
print(result)


[Document(page_content='languages, such as Chinese.\nIn this technical report, we introduce Baichuan\n2, a series of large-scale multilingual language\nmodels. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters. Both\nmodels were trained on 2.6 trillion tokens, which\nto our knowledge is the largest to date, more than\ndouble that of Baichuan 1 (Baichuan, 2023b,a).\nWith such a massive amount of training data,', metadata={'page': 1, 'source': '/tmp/tmphxnlz_0y/tmp.pdf'}), Document(page_content='Baichuan 2: Open Large-scale Language Models\nAiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan\nDian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai\nGuosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji\nJian Xie, Juntao Dai, Kun Fang, Lei Su, Liang Song, Lifeng Liu, Liyun Ru, Luyao Ma\nMang Wang, Mickel Liu, MingAn Lin, Nuolan Nie, Peidong Gu

通过本地部署的模型进行交互